# Import inference graph and extend it to a training graph:

To extend the graph, we have to add:
    - input data structure
    - loss function
    - optimization op
    
To build the graph it is necessary to know the node names of all relevant 
layers, placeholders etc. for restoring the model later.

pay ATTENTION to:
    the imported model must be in the same graph as the nodes which are 
    added later
    -> load the model first to the default graph, then add further ops
    
also test to input input-images of different sizes (multiples of 32px). 
It might not work because input placeholder is defined fix.
perhaps input node should not be saved inside the model?!

output of model has name (deconv_s2out_shading/BiasAdd:0 and 
deconv_s2out_albedo/BiasAdd:0). 
simpler names?!


To plot all graphs directly in this notebook, run jupyter form terminal like 
this:
    jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000

In [1]:
import os   
import sys
sys.path.append('./util')
import time
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import input_queues as iq
import cnn_model
import plot_helpers as plt_help
import general_helpers as ghelp
import nn_helpers as nnhelp
import cnn_helpers as cnnhelp

# make only 'gpu:0' visible, so that only one gpu is used not both, see also
# https://github.com/tensorflow/tensorflow/issues/5066
# https://github.com/tensorflow/tensorflow/issues/3644#issuecomment-237631171
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"


def train_network(LOGS_PATH, DATA_DIR, path_inference_graph, path_restore_model,
                  IMAGE_SHAPE, INITIAL_LEARNING_RATE, loss_opt, BATCH_SIZE, 
                  NUM_EPOCHS, DISPLAY_STEP, SAVE_STEP, 
                  plot_inference_graph=False, is_sample_size=False):
    """
    :param LOGS_PATH: path to summary files
    :param DATA_DIR: data path
    :param path_inference_graph:
    :param path_restore_model: e.g. 'logs/2/tfmodel-5' or None
    :param IMAGE_SHAPE: complete image size [436, 1024, 3]
        Narihira2015 use [M*32=13*32=416, 416, 3]
    :param INITIAL_LEARNING_RATE: hyper-parameters
    :param loss_opt:
    :param BATCH_SIZE: nr of data which is put through the network before 
        updating it, as default use: 16, 32 or 64. 
        BATCH_SIZE determines how many data samples are loaded in the memory 
        (be careful with memory space)
    :param NUM_EPOCHS: nr of times the training process loops through the 
        complete training data set (how often is the tr set 'seen')
        if you have 1000 training examples, and your batch size is 500, then it
        will take 2 iterations to complete 1 epoch.
    :param DISPLAY_STEP: every DIPLAY_STEP'th training iteration information is 
        printed (default: 100)
    :param SAVE_STEP: every SAVE_STEP'th training iteration a summary file is 
        written to LOGS_PATH and checkpoint files are saved
    :param plot_inference_graph: flag, True if inference graph should be 
        plotted (default: False).
    :param is_sample_size: flag, if True only a smaller sample size is used for 
        training and validation.
    """
    ############################################################################

    # create logger (write to file and stdout):
    logger = ghelp.create_logger(filename=LOGS_PATH + 'training.log')
    logger.debug('Python version: \n    ' + sys.version + 
                 '\n    Tensorflow version: ' + tf.__version__)
    logger.info('Training on images of shape: {}'.format(IMAGE_SHAPE))
    logger.info('Initial learning rate: {}'.format(INITIAL_LEARNING_RATE))
    logger.info('Loss function used for optimization: {}'.format(loss_opt))
    logger.info('Batch size: {}'.format(BATCH_SIZE))
    logger.info('# epochs: {}'.format(NUM_EPOCHS))
    logger.info('Report training set loss every {}'.format(DISPLAY_STEP) +
                'iteration.')
    logger.info('Write summary and checkpoints to file every ' +
                '{}-th iteration.'.format(SAVE_STEP))

    # load meta graph (inference graph)
    # how to work with restored models:
    # https://www.tensorflow.org/programmers_guide/meta_graph
    # http://cv-tricks.com/tensorflow-tutorial/save-restore-tensorflow-models-quick-complete-tutorial/
    saver_restore = tf.train.import_meta_graph(path_inference_graph, 
                                               clear_devices=True)
    logger.debug('Restored inference graph from\n' +
                 '    {}'.format(path_inference_graph))

    ############################################################################

    # save default graph in variable:
    graph = tf.get_default_graph()
    if plot_inference_graph:
        # plot imported inference graph:
        plt_help.show_graph(graph.as_graph_def())

    # lets get the input
    x = graph.get_tensor_by_name(name='input:0')

    # setup target output classes (ground truth):
    y_albedo_label = tf.placeholder(dtype=tf.float32, 
                                    shape=[None] + IMAGE_SHAPE, 
                                    name='out_albedo')
    y_shading_label = tf.placeholder(dtype=tf.float32, 
                                     shape=[None] + IMAGE_SHAPE, 
                                     name='out_shading')

    # bool variable that indicates if we are in training mode (training=True) or
    # valid/test mode (training=False) this indicator is important if dropout 
    # or/and batch normalization is used.
    training = graph.get_tensor_by_name(name='is_training:0')

    invalid_px_mask = tf.placeholder(dtype=tf.float32, 
                                     shape=[None] + IMAGE_SHAPE, 
                                     name='invalid_px_mask')

    # get graph output nodes:
    y_albedo_pred = graph.get_tensor_by_name(name='deconv_s2out_albedo/BiasAdd:0')
    y_shading_pred = graph.get_tensor_by_name(name='deconv_s2out_shading/BiasAdd:0')
    # y_albedo_pred = tf.clip_by_value(t=y_albedo, clip_value_min=0, 
    #                                  clip_value_max=1, 
    #                                  name='0_1_clipping_albedo')
    # y_shading_pred = tf.clip_by_value(t=y_shading, clip_value_min=0,
    #                                   clip_value_max=1, 
    #                                   name='0_1_clipping_shading')

    ############################################################################
    ############################################################################

    with tf.name_scope('loss'):
        valid_mask = cnnhelp.get_valid_pixels(image=x, 
                                              invalid_mask=invalid_px_mask)
        loss_type, lambda_ = 'berhu', None
        loss_berhu = cnnhelp.loss_fct(label_albedo=y_albedo_label,
                                      label_shading=y_shading_label,
                                      prediction_albedo=y_albedo_pred, 
                                      prediction_shading=y_shading_pred,
                                      lambda_=lambda_,
                                      loss_type=loss_type, 
                                      valid_mask=valid_mask, 
                                      log=True)

        loss_type, lambda_ = 'mse', 0
        loss_l2 = cnnhelp.loss_fct(label_albedo=y_albedo_label,
                                   label_shading=y_shading_label,
                                   prediction_albedo=y_albedo_pred, 
                                   prediction_shading=y_shading_pred,
                                   lambda_=lambda_,
                                   loss_type=loss_type, 
                                   valid_mask=valid_mask, 
                                   log=True)

        loss_type, lambda_ = 'mse', 1
        loss_l2_invariant = cnnhelp.loss_fct(label_albedo=y_albedo_label,
                                             label_shading=y_shading_label,
                                             prediction_albedo=y_albedo_pred, 
                                             prediction_shading=y_shading_pred,
                                             lambda_=lambda_,
                                             loss_type=loss_type, 
                                             valid_mask=valid_mask, 
                                             log=True)

        loss_type, lambda_ = 'mse', 0.5
        loss_l2_avg = cnnhelp.loss_fct(label_albedo=y_albedo_label,
                                       label_shading=y_shading_label,
                                       prediction_albedo=y_albedo_pred, 
                                       prediction_shading=y_shading_pred,
                                       lambda_=lambda_,
                                       loss_type=loss_type, 
                                       valid_mask=valid_mask, 
                                       log=True)

        loss_dict = {'berhu': loss_berhu,
                     'l2': loss_l2,
                     'l2_invariant': loss_l2_invariant,
                     'l2_avg': loss_l2_avg}

        loss = loss_dict[loss_opt]
    logger.debug('Defined losses.')

    ############################################################################

    # Use an AdamOptimizer to train the network:
    with tf.name_scope('optimization'):
        opt_step = tf.train.AdamOptimizer(INITIAL_LEARNING_RATE).minimize(loss)
    logger.debug('Defined optimization method.')

    ############################################################################

    # to get every summary defined above we merge them to get one target:
    merge_train_summaries = tf.summary.merge_all()
    # define a FileWriter op which writes summaries defined above to disk:
    summary_writer = tf.summary.FileWriter(LOGS_PATH)
    # Create a saver for writing training checkpoints.
    saver = tf.train.Saver(max_to_keep=NUM_EPOCHS)

    ############################################################################

    # introduce some validation set specific summaries
    # These summaries need to be defined blow the function 
    # merge_train_summaries = tf.summary.merge_all()
    # because the validation set summaries are added to the summary writer at 
    # different times. If they had been summarized with the training summaries 
    # they would have to be defined at times where merge_train_summaries are 
    # added to the summary writer
    with tf.name_scope('loss/valid/'):
        valid_dict = {key: tf.placeholder(dtype=tf.float32, name=key) for \
                      key in loss_dict.keys()}
        valid_summaries = [tf.summary.scalar(name=key, tensor=val) for \
                           key, val in valid_dict.items()]
        valid_sums_merged = tf.summary.merge(inputs=valid_summaries,
                                             collections=None, name=None)
    logger.debug('Defined validation losses.')
    logger.debug('Finished building training graph.')
    logger.info('Total parameters of network: ' +
                '{}'.format(nnhelp.network_params()))

    ############################################################################
    ############################################################################

    # import data:
    if is_sample_size:
        sample = 'sample_'
    else:
        sample = ''
    # import training data:
    file = sample + 'data_sintel_shading_train.csv'
    df_train = pd.read_csv(DATA_DIR + file, sep=',', header=None,
                           names=['img', 'alb', 'shad', 'invalid'])
    # complete image paths:
    df_train = DATA_DIR + df_train

    # # enable this line to train on only one image:
    # df_train1 = df_train.loc[[0]]
    # # replicate this row 100 times:
    # df_train = pd.concat([df_train1]*100).reset_index(drop=True)

    # instantiate a data queue for feeding data in (mini) batches to cnn:
    data_train = iq.DataQueue(df=df_train, batch_size=BATCH_SIZE,
                              num_epochs=NUM_EPOCHS)
    logger.debug('Imported training data ' + 
                 '(#: {}) '.format(data_train.df.shape[0]) + 
                 'from\n    {}'.format(DATA_DIR + file))

    # import validation data set: 
    # why not using the whole validation set for validation at once? 
    # - limited memory space.
    #  -> After each training epoch we will use the complete validation dataset
    #     to calculate the error/accuracy on the validation set
    file = sample + 'data_sintel_shading_valid.csv'
    df_valid = pd.read_csv(DATA_DIR + file, sep=',', header=None,
                           names=['img', 'alb', 'shad', 'invalid'])
    # complete image paths:
    df_valid = DATA_DIR + df_valid
    # instantiate a data queue for feeding data in (mini) batches to cnn:
    data_valid = iq.DataQueue(df=df_valid, batch_size=BATCH_SIZE,
                              num_epochs=NUM_EPOCHS)
    logger.debug('Imported validation data ' +
                 '(#: {}) '.format(data_valid.df.shape[0]) + 
                 'from\n    {}'.format(DATA_DIR + file))

    ############################################################################
    ############################################################################

    logger.info('Start training:')
    # Initialization:
    # Op that initializes global variables in the graph:
    init_global = tf.global_variables_initializer()
    # Op that initializes local variables in the graph:
    init_local = tf.local_variables_initializer()

    config = tf.ConfigProto(device_count = {'GPU': 1},
                            intra_op_parallelism_threads=3
    #                        allow_soft_placement = True,
    #                        intra_op_parallelism_threads=3,
    #                        log_device_placement=False
                           )
    with tf.Session(config=config) as sess: 
        ########################################################################
        # initialize all variables:
        sess.run([init_global, init_local])

        if path_restore_model:
            # restore saved model parameters (weights, biases, etc):
            saver_restore.restore(sess, path_restore_model)
            logger.info('Restoring parameters from ' +
                        '{}'.format(path_restore_model))

        # Adds a Graph to the event file.
        # create summary that give output (TensorFlow op that output protocol 
        # buffers containing 'summarized' data) of the built Tensorflow graph:
        summary_writer.add_graph(sess.graph)

        # start timer for total training time:
        start_total_time = time.time()
        # set timer to measure the displayed training steps:
        start_time = start_total_time

        ########################################################################
        
        # Training:
        # train loop
        # train until all data is processed (queue empty),
        # number of iterations depends on number of data, number of epochs and 
        # batch size:
        iter_start = data_train.iter_left
        logger.info('For training it takes {}\n'.format(iter_start) +
                    '    (= # data / batch_size * epochs) iterations to loop ' +
                    'through {} samples of\n  '.format(data_train.df.shape[0]) +
                    '  training data over {} '.format(data_train.num_epochs) +
                    'epochs summarized in batches of size ' + 
                    '{}.\n'.format(data_train.batch_size) +
                    '    So, there are # data / batch_size = ' +
                    '{}'.format(int(data_train.df.shape[0]/data_train.batch_size))+
                    ' iterations per epoch.')

        while data_train.iter_left >= 0:
            try:
                # take a (mini) batch of the training data:
                deq_train = data_train.dequeue()
                img_b, alb_b, shad_b, inv_b = iq.next_batch(deq=deq_train, 
                                                            output_shape=IMAGE_SHAPE,
                                                            is_scale=True,
                                                            is_flip=True, 
                                                            is_rotated=True,
                                                            norm=True)
                # run training/optimization step:
                # Run one step of the model.  The return values are the 
                # activations from the `train_op` (which is discarded) and the 
                # `loss` Op.  To inspect the values of your Ops or variables, 
                # you may include them in the list passed to sess.run() and the 
                # value tensors will be returned in the tuple from the call.
                feed_dict_tr = {x: img_b,
                                y_albedo_label: alb_b,
                                y_shading_label: shad_b,
                                invalid_px_mask: inv_b,
                                training: True}
                sess.run(opt_step, feed_dict=feed_dict_tr)

                ################################################################

                # report training set accuracy every DISPLAY_STEP-th step:
                if (data_train.num_iter) % DISPLAY_STEP == 0:
                    # console output:
                    train_loss_dict = {}
                    for key, val in loss_dict.items():
                        train_loss_dict[key] = sess.run(val, 
                                                        feed_dict=feed_dict_tr)

                    dur_time = time.time() - start_time
                    dur_time = ghelp.get_time_format(time_in_sec=dur_time)
                    dur_time = ghelp.time_tuple_to_str(time_tuple=dur_time)
                    logger.info('step {}: '.format(data_train.num_iter) +
                                'training ({}) loss '.format(loss_opt) +
                                '{:.4f}'.format(train_loss_dict[loss_opt]) + 
                                '\n    (' +
                                ', '.join(['{}: {:.4f}'.format(it[0], it[1]) \
                                           for it in train_loss_dict.items() \
                                           if it[0]!=loss_opt]) +
                                ' , ET: {}).'.format(dur_time))
                    # reset timer to measure the displayed training steps:
                    start_time = time.time()

                ################################################################

                # Validation:
                # display validation set accuracy and loss after each completed 
                # epoch (1 epoch ^= data_train.df.shape[0]/data_train.batch_size
                # training steps => steps per epoch)
                val_epoch = int(data_train.df.shape[0] / data_train.batch_size)
                
                if data_train.num_iter % val_epoch == 0:
                    # After each training epoch we will use the complete 
                    # validation data set to calculate the error/accuracy on the
                    # validation set:
                    # loop through one validation data set epoch:
                    # initialize dictionary which contains all validation 
                    # losses:
                    valid_loss_dict = dict.fromkeys(loss_dict, 0)
                    valid_steps_per_epoch = int(data_valid.df.shape[0] / 
                                                data_valid.batch_size)
                    for j in range(valid_steps_per_epoch):
                        # DISCLAIMER: we do not run the opt_step here (on 
                        # the validation data set) because we do not want to 
                        # train our network on the validation set. Important 
                        # for batch normalization and dropout 
                        # (training -> False).
                        # get validation data set (mini) batch:
                        lst = iq.next_batch(deq=data_valid.dequeue(), 
                                            output_shape=IMAGE_SHAPE,
                                            is_scale=False,
                                            is_flip=False,
                                            is_rotated=False,
                                            norm=True)
                        img_b_val, alb_b_val, shad_b_val, inv_b_val = lst

                        # calculate the mean loss of this validation batch and 
                        # sum it with the previous mean batch losses:
                        fd_val = {x: img_b_val,
                                         y_albedo_label: alb_b_val,
                                         y_shading_label: shad_b_val,
                                         invalid_px_mask: inv_b_val,
                                         training: False}

                        for key, val in loss_dict.items():
                            # divide each loss loss by the iteration steps 
                            # (steps_per_epoch) to get the mean val loss:
                            mean_val = val / valid_steps_per_epoch
                            valid_loss_dict[key] += sess.run(mean_val, 
                                                             feed_dict=fd_val)
                    # adding a mean loss summary op (for tensorboard):
                    feed_dict_vl = {valid_dict[key]: valid_loss_dict[key] for \
                                    key in valid_dict.keys()}
                    val_loss_sums = sess.run(valid_sums_merged,
                                             feed_dict=feed_dict_vl)
                    summary_writer.add_summary(summary=val_loss_sums, 
                                               global_step=data_train.num_iter)

                    dur_time = time.time() - start_time
                    dur_time = ghelp.get_time_format(time_in_sec=dur_time)
                    dur_time = ghelp.time_tuple_to_str(time_tuple=dur_time)
                    logger.info('step {} '.format(data_train.num_iter) +
                                '(epoch ' + 
                                '{}):'.format(data_train.completed_epochs + 1) +
                                ' mean validation losses:\n    ' +
                                ', '.join(['{}: {:.4f}'.format(it[0], it[1]) \
                                           for it in valid_loss_dict.items()]) +
                                ' (ET: {}).'.format(dur_time))
                    # reset timer to measure the displayed training steps:
                    start_time = time.time()

                ################################################################

                if data_train.num_iter % SAVE_STEP == 0:
                    # save checkpoint files to disk:
                    save_path = saver.save(sess, LOGS_PATH + 'tfmodel',
                                           global_step=data_train.num_iter)
                    s = sess.run(merge_train_summaries, feed_dict=feed_dict_tr)
                    # adds a Summary protocol buffer to the event file 
                    # (global_step: Number. Optional global step value to record
                    # with the summary. Each stepp i is assigned to the 
                    # corresponding summary parameter.)
                    summary_writer.add_summary(summary=s, 
                                               global_step=data_train.num_iter)
                    dur_time = time.time() - start_time
                    dur_time = ghelp.get_time_format(time_in_sec=dur_time)
                    dur_time = ghelp.time_tuple_to_str(time_tuple=dur_time)
                    logger.info('Saved data (step ' + 
                                '{}):\n'.format(data_train.num_iter) +
                                '    Checkpoint file written to: ' + 
                                '{} '.format(save_path) +
                                '(ET: {}).'.format(dur_time))
                    # reset timer to measure the displayed training steps:
                    start_time = time.time()

            # end while loop when there are no elements left to dequeue:
            except IndexError:
                end_total_time = time.time() - start_total_time
                end_total_time = ghelp.get_time_format(end_total_time)
                end_total_time = ghelp.time_tuple_to_str(time_tuple=end_total_time)
                logger.info('Training done... total training time: ' + 
                            '{}.'.format(end_total_time))
                break

    logger.info('Finished training.')

In [2]:
m_height = 13  # multiplicate of image height size -> network is designed so 
    # that it can take images with shape of multiples of m
m_width = m_height+19  # multiplicate of image width size -> network 
    # is designed so that it can take images with shape of multiples of m

params = {'LOGS_PATH': 'logs/3/',  # path to summary files
          'DATA_DIR': '/usr/udo/data/',
          'path_inference_graph': 'models/narihira2015/tfmodel_inference.meta',
          'path_restore_model': 'logs/2/tfmodel-84', # e.g. 'logs/2/tfmodel-5' or None
          'IMAGE_SHAPE': [32 * m_height, 32 * m_width, 3],
          'INITIAL_LEARNING_RATE': 5e-4,  # hyper param
          'loss_opt': 'berhu',
          'BATCH_SIZE': 8,  # hyper param
          'NUM_EPOCHS': 2,  # hyper param 
          'DISPLAY_STEP': 2,
          'SAVE_STEP': 5,
          'plot_inference_graph': False,
          'is_sample_size': True
         }

In [ ]:
train_network(**params)

[2017-09-10 17:00:00 DEBUG]: Python version: 
    3.5.0+ (default, Oct 11 2015, 09:05:38) 
[GCC 5.2.1 20151010]
    Tensorflow version: 1.2.0
[2017-09-10 17:00:00 INFO]: Training on images of shape: [416, 1024, 3]
[2017-09-10 17:00:00 INFO]: Initial learning rate: 0.0005
[2017-09-10 17:00:00 INFO]: Loss function used for optimization: berhu
[2017-09-10 17:00:00 INFO]: Batch size: 8
[2017-09-10 17:00:00 INFO]: # epochs: 2
[2017-09-10 17:00:00 INFO]: Report training set loss every 2iteration.
[2017-09-10 17:00:00 INFO]: Write summary and checkpoints to file every 5-th iteration.
[2017-09-10 17:00:01 DEBUG]: Restored inference graph from
    models/narihira2015/tfmodel_inference.meta
[2017-09-10 17:00:01 DEBUG]: Defined losses.
[2017-09-10 17:00:03 DEBUG]: Defined optimization method.
[2017-09-10 17:00:03 DEBUG]: Defined validation losses.
[2017-09-10 17:00:03 DEBUG]: Finished building training graph.
[2017-09-10 17:00:03 INFO]: Total parameters of network: 20533190
[2017-09-10 17:00:03 D

INFO:tensorflow:Restoring parameters from logs/2/tfmodel-84


[2017-09-10 17:00:04 INFO]: Restoring parameters from logs/2/tfmodel-84
[2017-09-10 17:00:05 INFO]: For training it takes 12
    (= # data / batch_size * epochs) iterations to loop through 50 samples of
    training data over 2 epochs summarized in batches of size 8.
    So, there are # data / batch_size = 6 iterations per epoch.
[2017-09-10 17:00:31 INFO]: step 2: training (berhu) loss 0.2944
    (l2: 0.0756, l2_avg: 0.0746, l2_invariant: 0.0736 , ET: 26 sec).
[2017-09-10 17:00:49 INFO]: step 4: training (berhu) loss 0.2598
    (l2: 0.0668, l2_avg: 0.0650, l2_invariant: 0.0633 , ET: 18 sec).
[2017-09-10 17:01:04 INFO]: Saved data (step 5):
    Checkpoint file written to: logs/3/tfmodel-5 (ET: 14 sec).
[2017-09-10 17:01:13 INFO]: step 6: training (berhu) loss 0.3022
    (l2: 0.0784, l2_avg: 0.0749, l2_invariant: 0.0715 , ET: 9 sec).
[2017-09-10 17:01:18 INFO]: step 6 (epoch 1): mean validation losses:
    berhu: 0.6068, l2: 0.2282, l2_avg: 0.2118, l2_invariant: 0.1953 (ET: 5 sec).


In [16]:
# !tensorboard --logdir /logs/3

Starting TensorBoard b'54' at http://titanx2:6006
(Press CTRL+C to quit)
^C
